In [ ]:
import pandas as pd
import tkinter as tk
import warnings
import locale
import os
from tkinter import messagebox
from datetime import datetime

warnings.filterwarnings('ignore')
locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')

hoy = datetime.now()
año = hoy.year
mes = hoy.month 

mes_nombre = hoy.strftime('%b').upper()[:3] # ENE
mes_año = f'{mes_nombre}{str(año)[2:]}' # ENE24
fecha = f'{año}{str(mes).zfill(2)}' # 202401

# Datos asignacion
asignacion_camp_path = f'bases/asignacion/{fecha}/base_asignacion_{mes_año}_CAMP.txt'

# Datos campaña
contactos_path = f'bases/campañas/{fecha}/CONTACTOS_AGENCIAS.xlsx'
campañas_path = f'bases/campañas/{fecha}/CAMPAÑAS_{mes_año}.xlsx'
final_path = os.path.abspath(campañas_path)

# Datos campaña impago
base_impago_path = f'bases/campañas/{fecha}/BASE_CONDONACION_{fecha}.xlsx'
impago_path = os.path.abspath(base_impago_path)

print('Base Asignacion:', asignacion_camp_path)
print('Base Campañas:', campañas_path)
print('Base Contactos:', contactos_path)
print('Base Impago:', base_impago_path)

In [13]:
def clean_columns(columns_list: list[str]) -> list[str]:
    return [column.strip().replace('.', '').replace(' ', '_').upper() for column in columns_list]

In [ ]:
root = tk.Tk()
root.attributes('-topmost', True)
root.withdraw()

result = messagebox.askquestion('Confirmación', '¿Cargar base asignacion?', icon='warning')

if result == 'yes':
    df_asignacion = pd.read_csv(asignacion_camp_path, sep='|', encoding='utf-8', dtype={'CC': str, 'CONTRATO': str})
    print('Base Asignacion:', df_asignacion.shape)
    print(df_asignacion.columns)

root.destroy()

In [ ]:
df_asignacion_test = df_asignacion.copy()
df_asignacion_test.columns = clean_columns(df_asignacion_test.columns)

cols_required = ['CC', 'CONTRATO', 'CARTERA', 'NOMBRE_CLIENTE', 'MONEDA', 'CAPITAL', 'CAMP', 'CAMPAÑA', 'TIPO_CARTERA', 'AGENCIA', 'TIPO_FONDO', 'OFICINA', 'TERRITORIO', 'ID_VTA']
df_asignacion_test = df_asignacion_test[cols_required]

print('Base Asignacion:', df_asignacion_test.shape)

if df_asignacion_test['CAMP'].dtype == 'object':
    df_asignacion_test['CAMP'] = df_asignacion_test['CAMP'].str.extract(r'(\d+)%').astype(float) / 100
df_asignacion_test['CAMP'] = pd.to_numeric(df_asignacion_test['CAMP'], errors='coerce')
df_asignacion_test['CAMP'].fillna(0, inplace=True)

df_asignacion_test.rename(columns={
    'CC': 'CODIGO CENTRAL', 
    'CONTRATO': 'N° CONTRATO', 
    'NOMBRE_CLIENTE': 'NOMBRE DEL CLIENTE', 
    'CAPITAL': 'DEUDA CAPITAL', 
    'CAMP': '% CONDONACIÓN', 
    'CAMPAÑA': 'MONTO A PAGAR', 
    'TIPO_CARTERA': 'TIPO CARTERA', 
    'AGENCIA': 'AGENCIA DE COBRANZA', 
    'TIPO_FONDO': 'TIPO DE FONDO', 
    'ID_VTA': 'ID_VENTA'
}, inplace=True)

df_asignacion_test['TIPO CARTERA'] = df_asignacion_test['TIPO CARTERA'].fillna('UNSECURED')
df_asignacion_test = df_asignacion_test[df_asignacion_test['TIPO CARTERA'] == 'UNSECURED']
df_asignacion_test['ID_VENTA'] = df_asignacion_test['ID_VENTA'].fillna('NULL')
df_asignacion_test = df_asignacion_test[df_asignacion_test['ID_VENTA'].isin(['NULL', '-'])]
df_asignacion_test = df_asignacion_test[~df_asignacion_test['AGENCIA DE COBRANZA'].isin(['SIN AGENCIA'])]

df_asignacion_test['DEUDA CAPITAL'] = df_asignacion_test['DEUDA CAPITAL'].round(2)
df_asignacion_test['% CONDONACIÓN'] = df_asignacion_test['% CONDONACIÓN'].round(2)
df_asignacion_test['MONTO A CONDONAR'] = (df_asignacion_test['DEUDA CAPITAL'] * df_asignacion_test['% CONDONACIÓN']).round(2)
df_asignacion_test['TIPO DE FONDO'] = df_asignacion_test['TIPO DE FONDO'].str.strip()
df_asignacion_test['TIPO DE FONDO'] = df_asignacion_test['TIPO DE FONDO'].fillna('NULL')
df_asignacion_test['MONTO A PAGAR'] = df_asignacion_test['DEUDA CAPITAL'] - df_asignacion_test['MONTO A CONDONAR']
df_asignacion_test['MONTO A PAGAR'] = df_asignacion_test['MONTO A PAGAR'].round(2)
df_asignacion_test['MONTO A PAGAR'] = df_asignacion_test.apply(lambda x: 'DEUDA TOTAL' if x['TIPO DE FONDO'] in ['REACTIVA', 'FAE', 'CRECER'] else x['MONTO A PAGAR'], axis=1)
df_asignacion_test['MONTO A PAGAR'] = df_asignacion_test['MONTO A PAGAR'].astype(str)
print('Base Asignacion:', df_asignacion_test.shape)

df_contactos = pd.read_excel(contactos_path)
df_asignacion_test = df_asignacion_test.merge(df_contactos, how='left', left_on=['CARTERA', 'AGENCIA DE COBRANZA'], right_on=['CARTERA', 'AGENCIA'])
print('Base Asignacion:', df_asignacion_test.shape)

df_asignacion_test.rename(columns={
    'CORREO': 'CORREO DE LA AGENCIA', 
    'TELEFONO': 'TELEFONO DE AGENCIA'
}, inplace=True)

df_asignacion_test['AGENCIA DE COBRANZA'] = df_asignacion_test['AGENCIA DE COBRANZA'].str.upper()
df_asignacion_test['CORREO DE LA AGENCIA'] = df_asignacion_test['CORREO DE LA AGENCIA'].str.upper()
df_asignacion_test['CONTACTO BBVA'] = df_asignacion_test['CONTACTO BBVA'].str.upper()
df_asignacion_test['CORREO BBVA'] = df_asignacion_test['CORREO BBVA'].str.upper()

df_asignacion_test.sort_values(by=['CARTERA', 'CODIGO CENTRAL', '% CONDONACIÓN'], inplace=True)
df_asignacion_test.reset_index(drop=True, inplace=True)

cols_final = ['CODIGO CENTRAL', 'N° CONTRATO', 'CARTERA', 'NOMBRE DEL CLIENTE', 'MONEDA', 'DEUDA CAPITAL', '% CONDONACIÓN', 'MONTO A CONDONAR', 'MONTO A PAGAR', 'AGENCIA DE COBRANZA', 'TIPO DE FONDO', 'TELEFONO DE AGENCIA', 'CORREO DE LA AGENCIA', 'CONTACTO BBVA', 'CORREO BBVA' , 'TELEFONO BBVA', 'OFICINA', 'TERRITORIO']
df_asignacion_test = df_asignacion_test[cols_final]

print('Base Campañas:', df_asignacion_test.shape)
df_asignacion_test.head(5)

In [26]:
import openpyxl as op
from openpyxl.styles import Font, PatternFill, Alignment

def format_excel(file_path: str) -> None:
    workbook = op.load_workbook(file_path)
    sheet = workbook.active
    
    general_font = Font(name='Calibri', size=11)
    header_font = Font(name='Calibri', size=11, bold=True, color='FFFFFF')
    alignment_center = Alignment(horizontal='center', vertical='center')
    
    header_fill_blue = PatternFill(start_color='0F243E', end_color='0F243E', fill_type='solid')
    
    sheet.row_dimensions[1].height = 50
    
    for row in sheet.iter_rows():
        for cell in row:
            cell.font = general_font
    
    for cell in sheet[1]:
        cell.font = header_font
        cell.alignment = alignment_center
    
    for col in range(1, 19):  # Columnas A-R
        sheet.cell(row=1, column=col).fill = header_fill_blue
    
    for column in sheet.columns:
        max_length = 0
        column_letter = column[0].column_letter
        for cell in column:
            try:
                if len(str(cell.value)) > max_length:
                    max_length = len(cell.value)
            except:
                pass
        adjusted_width = (max_length + 2)
        sheet.column_dimensions[column_letter].width = adjusted_width
    
    workbook.save(file_path)

In [27]:
root = tk.Tk()
root.attributes('-topmost', True)
root.withdraw()

result = messagebox.askquestion('Confirmación', '¿Cargar base impago?', icon='warning')
if result == 'yes':
    df_impago = pd.read_excel(base_impago_path, sheet_name='DATA')
    
    df_impago_test = df_impago.copy()
    print(df_impago_test.columns)
    print('Base Impago:', df_impago_test.shape)
    
    df_impago_test.columns = clean_columns(df_impago_test.columns)
    cols_required = ['CODCENTRAL', 'CONTRATO', 'CLIENTE', 'MONEDA', 'DIRECTA', 'TASA_DSCTO', 'MONTOPAGO', 'ENTIDAD_COB', 'OFICINA', 'TERRITORIO', 'SEGMENTO']
    df_impago_test = df_impago_test[cols_required]
    print('Base Impago:', df_impago_test.shape)
    
    df_impago_test.rename(columns={
        'CODCENTRAL': 'CODIGO CENTRAL', 
        'CONTRATO': 'N° CONTRATO', 
        'CLIENTE': 'NOMBRE DEL CLIENTE', 
        'DIRECTA': 'DEUDA CAPITAL', 
        'TASA_DSCTO': '% CONDONACIÓN', 
        'MONTOPAGO': 'MONTO A PAGAR', 
        'ENTIDAD_COB': 'AGENCIA DE COBRANZA', 
    }, inplace=True)
    
    df_impago_test['CARTERA'] = 'IMPAGO'
    df_impago_test['TIPO DE FONDO'] = 'NULL'
    
    df_impago_test['CODIGO CENTRAL'] = df_impago_test['CODIGO CENTRAL'].astype(str).astype('Int64').astype(str).str.zfill(8)
    df_impago_test['N° CONTRATO'] = df_impago_test['N° CONTRATO'].apply(lambda x: str(int(x)).zfill(18) if pd.notna(x) else x)
    df_impago_test['MONTO A CONDONAR'] = df_impago_test['DEUDA CAPITAL'] * df_impago_test['% CONDONACIÓN']
    df_impago_test['TERRITORIO'] = df_impago_test['TERRITORIO'].fillna('NULL').astype(str)
    
    correos = {
        ('RECSA', 'PARTICULARES'): 'alfonso.moises@recsa.com / jean.yanqui@recsa.com',
        ('RECSA', 'PARTICULARES CONVENIOS'): 'alfonso.moises@recsa.com / jean.yanqui@recsa.com',
        ('RECSA', 'PYME'): 'sandra.clavo@recsa.com / katiuska.vera@recsa.com',
        ('ATENTO', 'PARTICULARES'): 'cvillarj@atento.com',
        ('ATENTO', 'PARTICULARES CONVENIOS'): 'cvillarj@atento.com',
        ('ATENTO', 'PYME'): 'jtumialanp@atento.com',
        ('ASESCOM RJ', 'PARTICULARES'): 'virginia.sifuentes@rjabogados.com',
        ('ASESCOM RJ', 'PARTICULARES CONVENIOS'): 'virginia.sifuentes@rjabogados.com',
        ('ASESCOM RJ', 'PYME'): 'bety.cabrera@rjabogados.com',
    }
    
    telefonos = {
        ('RECSA', 'PARTICULARES'): '01 7010519 / 961370088',
        ('RECSA', 'PARTICULARES CONVENIOS'): '01 7010519 / 961370088',
        ('RECSA', 'PYME'): '01 7099302',
        ('ATENTO', 'PARTICULARES'): '996044852 / 990384855 / 967683768',
        ('ATENTO', 'PARTICULARES CONVENIOS'): '996044852 / 990384855 / 967683768',
        ('ATENTO', 'PYME'): '987708366',
        ('ASESCOM RJ', 'PARTICULARES'): '938562012',
        ('ASESCOM RJ', 'PARTICULARES CONVENIOS'): '938562012',
        ('ASESCOM RJ', 'PYME'): '908874634',
    }
    
    def asignar_correo(row):
        return correos.get((row['AGENCIA DE COBRANZA'], row['SEGMENTO']), correos.get((row['AGENCIA DE COBRANZA'], None), None))
    
    def asignar_telefono(row):
        return telefonos.get((row['AGENCIA DE COBRANZA'], row['SEGMENTO']), telefonos.get((row['AGENCIA DE COBRANZA'], None), None))
    
    df_impago_test['CORREO DE LA AGENCIA'] = df_impago_test.apply(asignar_correo, axis=1)
    df_impago_test['TELÉFONO DE AGENCIA'] = df_impago_test.apply(asignar_telefono, axis=1)
    
    df_impago_test['CONTACTO BBVA'] = 'recuperaciones@opplus.bbva.com'
    df_impago_test['CORREO BBVAA'] = 'recuperaciones@opplus.bbva.com'
    df_impago_test['TELÉFONO'] = '595-0030'
    
    df_impago_test.sort_values(by=['CARTERA', 'CODIGO CENTRAL', '% CONDONACIÓN'], inplace=True)
    df_impago_test.reset_index(drop=True, inplace=True)
    
    cols_final = ['CODIGO CENTRAL', 'N° CONTRATO', 'CARTERA', 'NOMBRE DEL CLIENTE', 'MONEDA', 'DEUDA CAPITAL', '% CONDONACIÓN', 'MONTO A CONDONAR', 'MONTO A PAGAR', 'AGENCIA DE COBRANZA', 'TIPO DE FONDO', 'TELÉFONO DE AGENCIA', 'CORREO DE LA AGENCIA', 'CONTACTO BBVA', 'CORREO BBVAA' , 'TELÉFONO', 'OFICINA', 'TERRITORIO']
    df_impago_test = df_impago_test[cols_final]
    
    print('Campaña Impago:', df_impago_test.shape)
    
    df_camp_final = pd.concat([df_asignacion_test, df_impago_test])
    print('Campaña total:', df_camp_final.shape)
    
    df_camp_final['CODIGO CENTRAL'] = df_camp_final['CODIGO CENTRAL'].astype(str).astype('Int64').astype(str).str.zfill(8)
    df_camp_final['N° CONTRATO'] = df_camp_final['N° CONTRATO'].apply(lambda x: str(int(x)).zfill(18) if pd.notna(x) else x)
    
    df_camp_final.to_excel(campañas_path, index=False, sheet_name='BASE')
else:
    df_asignacion_test.to_excel(campañas_path, index=False, sheet_name='BASE')

result = messagebox.askquestion('Confirmación', '¿Abrir archivo?', icon='warning')
if result == 'yes':
    os.startfile(final_path)

root.destroy()